In [1]:
import requests as r
from bs4 import BeautifulSoup as bs
import json 
from queue import Queue
import threading
import re
import time
import random
import os

In [210]:
#pre_url_str = 'http://www.ipeen.com.tw/shop/1055186-%E6%BF%83%E5%92%96%E5%95%A1-PEI-CAFE-%E5%90%8E%E9%87%8C'
#pre_url_str = 'http://www.ipeen.com.tw/shop/1071950-%E7%B4%85%E8%91%89%E8%9B%8B%E7%B3%95'
pre_url_str = 'http://www.ipeen.com.tw/shop/33714-%E5%92%96%E5%95%A1%E5%BC%84-%E5%BF%A0%E5%AD%9D%E5%BA%97'

In [211]:
restaurant_id = pre_url_str.split('/')[-1].split('-')[0]
restaurant_information_url = 'http://www.ipeen.com.tw/touch/shop.php?id={}'.format(restaurant_id)

In [212]:
restaurant_id

'33714'

In [213]:
restaurant_information_url

'http://www.ipeen.com.tw/touch/shop.php?id=33714'

In [214]:
phoneStartTime = time.time()
#test_restaurant_information_url = 'http://www.ipeen.com.tw/touch/shop.php?id=1055186'
#test_restaurant_information_url = 'http://www.ipeen.com.tw/touch/shop.php?id=1071950'
#test_restaurant_information_url = 'http://www.ipeen.com.tw/touch/shop.php?id=33714'
header1 = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
          'Accept-Encoding': 'gzip, deflate, sdch',
          'Accept-Language': 'zh-TW,zh;q=0.8,en-US;q=0.6,en;q=0.4',
          'Cache-Control': 'no-cache',
          'Connection': 'keep-alive',
          'Host': 'www.ipeen.com.tw',
          'Pragma': 'no-cache',
          'DNT': '1',
          'Upgrade-Insecure-Requests': '1',
          'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 9_1 like Mac OS X) AppleWebKit/601.1.46 (KHTML, like Gecko) \
          Version/9.0 Mobile/13B143 Safari/601.1'
          }
res_restaurant_information = r.get(restaurant_information_url, headers=header1)
print(res_restaurant_information.status_code)
soup_restaurant_information = bs(res_restaurant_information.text, 'lxml')
phoneEndTime = time.time()
print(phoneEndTime - phoneStartTime)

200
0.5760328769683838


In [209]:
soup_restaurant_information

<!DOCTYPE html>
<!--[if lt IE 8]><html class="no-js ie7" lang="zh-tw"><![endif]--><!--[if IE 8]><html class="no-js ie8" lang="zh-tw"><![endif]--><!--[if IE 9]><html class="no-js ie9" lang="zh-tw"><![endif]--><!--[if IE 10]><html class="no-js ie10" lang="zh-tw"><![endif]--><!--[if gt IE 10]><!--><html class="no-js" lang="zh-tw"><!--<![endif]-->
<head>
<meta charset="utf-8"/>
<title>濃咖啡 PEI CAFE(后里)@台中市后里區 - iPeen愛評網商家情報</title>
<link href="/images/touch/frame/touch-icon-iphone.png" rel="apple-touch-icon"/>
<link href="/images/touch/frame/touch-icon-ipad.png" rel="apple-touch-icon" sizes="76x76"/>
<link href="/images/touch/frame/touch-icon-iphone-retina.png" rel="apple-touch-icon" sizes="120x120"/>
<link href="/images/touch/frame/touch-icon-ipad-retina.png" rel="apple-touch-icon" sizes="152x152"/>
<meta content="濃咖啡 PEI CAFE是咖啡專賣，濃咖啡 PEI CAFE位於台中市后里區中山路99號，是咖啡、簡餐、茶的咖啡專賣。" name="description"/>
<meta content="濃咖啡 PEI CAFE(后里),咖啡、簡餐、茶,台中市,后里區" name="keywords"/>
<meta content="app-id=3664794

# 店名

In [215]:
restaurant_name = soup_restaurant_information.find('span', {'itemprop': 'name'}).text
restaurant_name

'咖啡弄'

# 餐廳地址與googleMapUrl

In [216]:
google_map_url = soup_restaurant_information.find('a', {'href': re.compile('(http:\/\/maps\.google\.com\/maps\?q=)')})['href']
print(google_map_url)

def addressRepl(matchobj):
    return matchobj.group(2)
restaurant_address = re.sub('(http:\/\/maps\.google\.com\/maps\?q=)(.+$)',addressRepl ,google_map_url)
print(restaurant_address)

http://maps.google.com/maps?q=台北市台北市大安區忠孝東路四段101巷45號
台北市台北市大安區忠孝東路四段101巷45號


# 電話

In [217]:
phone_pattarn = '(tel:)?(\(?\d{2}\)?)?(-|\s+)?(\d{3}|\d{4})(-|\s+)?(\d{4})'

In [218]:
phone_number = soup_restaurant_information.find('span', {'itemprop': 'telephone'}).text
if re.match(phone_pattarn, phone_number):
    print(phone_number)

02-2711-1912


# 標籤ul底下 四大標籤li出現順序為1.綜合評分 2.我的評分 3.(平均消費) > 餐廳類別

In [219]:
first_li_under_ul = soup_restaurant_information.find('li', {'itemprop': 'aggregateRating'})

# 綜合評分

In [220]:
aggregateRating_int = int(first_li_under_ul.find('span', {'itemprop': 'ratingValue'}).text)
if aggregateRating_int != 0:
    print(aggregateRating_int)
else:
    print('無有效綜合評分')

45


# 平均消費&餐廳類別

In [225]:
third_li_under_ul = first_li_under_ul.find_next_sibling().find_next_sibling()
if re.match('平均消費', third_li_under_ul.text):
    avg_price = int(third_li_under_ul.text.split('：')[1])
    print(avg_price)
    category = third_li_under_ul.find_next_sibling().text
    print(category)
else:
    print('沒有均消')
    category = third_li_under_ul.text
    print(category)

269
甜點專賣


# 營業時間&營業時間&官方網站&推薦菜&分類標籤

In [228]:
soup_restaurant_information.table

<table>
<tr>
<th>商家名稱</th>
<td>咖啡弄(忠孝店)                </td>
</tr>
<tr>
<th>電話</th>
<td>
<a class="ga_tracking" data-action="details_phone" data-category="MWEB_shop" data-label="詳細資訊-電話" href="tel:02-2711-1912">02-2711-1912</a> </td>
</tr>
<tr>
<th>地址</th>
<td>台北市大安區忠孝東路四段101巷45號            
                                            <a href="../touch/traffic.php?id=33714"><p>(詳細交通資訊)</p></a>
</td>
</tr>
<tr>
<th>捷運資訊</th>
<td>近台北市捷運 - 板南線 - 忠孝敦化站 (步行約6分鐘)<br/></td>
</tr>
<tr>
<th>公休日</th>
<td>無</td>
</tr>
<tr>
<th>營業時間</th>
<td>
                    今日 <span>12:00~23:00</span> <a class="business-hour-show-all" href="#">顯示全部</a>
<div class="business-hour-pannel">
<div class="business-hour-detail hide">
<table>
<tr class="today">
<th>週一</th>
<td>
                                                                                      12:00~23:00                                                      </td>
</tr>
<tr class="">
<th>週二</th>
<td>
                                                  

In [79]:
#判斷是否有食記分享
if soup_restaurant_information.find('div', {'id': 'comment'}):    
    print('有分享')    
    #取得總共有幾篇
    pre_comment_number_str = soup_restaurant_information.find('div', {'id': 'comment'}).h3.text
    def numberRepl(matchobj):
        return matchobj.group(2)
    comment_number = int(re.sub('(會員分享文\(共)(\d+)( 篇\))',numberRepl ,pre_comment_number_str))
    print(comment_number)
    
    #依照篇數，計算一共有幾頁
    if comment_number <= 5:
        int_page = 1
        print(int_page)
    elif comment_number %5 == 0:
        int_page=int(comment_number/5)
        print(int_page)
    else:
        int_page=int(comment_number/5)+1
        print(int_page)
    
    #產生分享文列表的全部頁數的url
    for i in range(1,int_page+1):
        comment_page_url = 'http://www.ipeen.com.tw/touch/cmmList.php?p={}&id={}'.format(i, restaurant_id)
        print(comment_page_url)
        #restaurant_share(comment_page_url)
else:
    print('尚無分享')

有分享
1
1
http://www.ipeen.com.tw/touch/cmmList.php?p=1&id=1071950


In [110]:
def restaurant_share(comment_page_url):
    
    try:
        res_comment_page = r.get(comment_page_url, headers=header1)
    except HTTPError:
            return None
    print(res_comment_page.status_code)
    res_comment_page.encoding="utf-8"
    soup_comment_page = bs(res_comment_page.text, 'lxml')
    
    
    pre_comment_url_list = soup_comment_page.findAll('ol')
    
    for pre_comment_url in pre_comment_url_list:
        this_pre_comment_url = pre_comment_url.parent['href']
        print(this_pre_comment_url)
        comment_id = this_pre_comment_url.split('=')[-1]
        print(comment_id)
        comment_url = 'http://www.ipeen.com.tw{}'.format(this_pre_comment_url)
        print(comment_url)
        comment_title = pre_comment_url.li.text
        print(comment_title)